In [ ]:
import pyrealsense2 as rs
import numpy as np
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO
from collections import deque

# ---------------------------
# 설정
# ---------------------------
bag_path  = r"/root/workspace/data/r3.bag"

# 파라미터
EPS_DEFAULT   = 0.01
ALPHA_CAMMOVE = 0.8
BETA_RATIO    = 1.0
SMOOTH_WINDOW = 8
STATIC_HOLD   = 4
DYNAMIC_REPORT_RANGE = 7.0

# ---------------------------
# RealSense 설정
# ---------------------------
pipeline = rs.pipeline()
config = rs.config()
config.enable_device_from_file(bag_path, repeat_playback=False)
config.enable_stream(rs.stream.depth, rs.format.z16, 30)
config.enable_stream(rs.stream.color, rs.format.rgb8, 30)
config.enable_stream(rs.stream.accel, rs.format.motion_xyz32f, 100)
config.enable_stream(rs.stream.gyro,  rs.format.motion_xyz32f, 200)

profile = pipeline.start(config)
align = rs.align(rs.stream.color)
depth_scale = profile.get_device().first_depth_sensor().get_depth_scale()

# ---------------------------
# YOLO 모델
# ---------------------------
model = YOLO(r"/workspace/yolo11m-seg.pt", task="segment")

# ---------------------------
# 전역 상태 변수
# ---------------------------
prev_time = None
velocity = np.zeros(2)
my_pos = np.zeros(2)
last_pos = np.zeros(2)
prev_depths = {}
smooth_buffers = {}
static_count = {}
frame_count = 0

# ---------------------------
# 프레임 처리 함수
# ---------------------------
def process_frame(frames):
    global prev_time, velocity, my_pos, last_pos, prev_depths, frame_count

    aligned = align.process(frames)
    dframe = aligned.get_depth_frame()
    cframe = aligned.get_color_frame()
    if not dframe or not cframe:
        return

    color = cv2.cvtColor(np.asanyarray(cframe.get_data()), cv2.COLOR_RGB2BGR)
    depth = np.asanyarray(dframe.get_data()) * depth_scale
    h0, w0 = color.shape[:2]

    # IMU 이동량 계산
    accel = None
    try:
        ms = frames.first_or_default(rs.stream.accel)
        if ms:
            accel = ms.as_motion_frame().get_motion_data()
    except Exception:
        for f in frames:
            if f.is_motion_frame() and f.get_profile().stream_type() == rs.stream.accel:
                accel = f.as_motion_frame().get_motion_data()
                break

    t = frames.get_timestamp() / 1000.0
    dt = 0 if prev_time is None else (t - prev_time)
    prev_time = t

    if accel is not None and dt > 0:
        v_acc = np.array([accel.x, accel.y])
        velocity[:] = np.clip(velocity + v_acc * dt, -1.0, 1.0)
        my_pos += velocity * dt

    cam_move = np.linalg.norm(my_pos - last_pos)
    last_pos = my_pos.copy()
    print(f"[Frame {frame_count}] cam_move={cam_move:.3f}")

    # YOLO 탐지 + 추적
    try:
        results = model.track(color, tracker="bytetrack.yaml", persist=True, conf=0.25)[0]
    except Exception as e:
        print(f"[Frame {frame_count}] YOLO error: {e}")
        return

    if getattr(results, "boxes", None) is None or getattr(results.boxes, "id", None) is None or len(results.boxes) == 0:
        frame_count += 1
        return

    boxes = results.boxes.data.cpu().numpy()
    ids = results.boxes.id.cpu().numpy().astype(int)

    # 객체별 깊이 계산
    cur_depths = {}
    for i, tid in enumerate(ids):
        x1, y1, x2, y2 = boxes[i][:4].astype(int)
        x1, y1 = max(0, x1), max(0, y1)
        x2, y2 = min(w0 - 1, x2), min(h0 - 1, y2)
        # ROI 중앙부만 사용
        cx1 = int(x1 + (x2 - x1) * 0.25)
        cx2 = int(x2 - (x2 - x1) * 0.25)
        cy1 = int(y1 + (y2 - y1) * 0.25)
        cy2 = int(y2 - (y2 - y1) * 0.25)
        roi = depth[cy1:cy2, cx1:cx2]
        vals = roi[roi > 0]
        if len(vals) == 0:
            continue
        median_d = float(np.median(vals))

        buf = smooth_buffers.setdefault(tid, deque(maxlen=SMOOTH_WINDOW))
        buf.append(median_d)
        smoothed_d = np.mean(buf)
        cur_depths[tid] = smoothed_d

    # 동적/정적 판정
    for tid, cur_d in cur_depths.items():
        prev_d = prev_depths.get(tid, cur_d)
        delta = prev_d - cur_d
        cam_adj = cam_move * ALPHA_CAMMOVE

        # 7m 이상이면 회색 박스만
        if cur_d > DYNAMIC_REPORT_RANGE:
            x1, y1, x2, y2 = boxes[list(ids).index(tid)][:4].astype(int)
            cv2.rectangle(color, (x1, y1), (x2, y2), (160, 160, 160), 2)
            print(f"    Obj#{tid}: {cur_d:.2f}m >7m → gray box only")
            continue

        # 비율 + 임계 혼합판정
        if abs(delta) > (cam_adj + EPS_DEFAULT) and abs(delta) > BETA_RATIO * (cam_adj + EPS_DEFAULT):
            state = "dynamic"
            static_count[tid] = 0
        else:
            static_count[tid] = static_count.get(tid, 0) + 1
            state = "static" if static_count[tid] >= STATIC_HOLD else "dynamic?"

        color_box = (0, 0, 255) if "dynamic" in state else (255, 0, 0)
        x1, y1, x2, y2 = boxes[list(ids).index(tid)][:4].astype(int)
        cv2.rectangle(color, (x1, y1), (x2, y2), color_box, 2)
        cv2.putText(color, f"{state} | Δ={delta:+.3f} | cam={cam_move:.3f}",
                    (x1, max(0, y1 - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.55, color_box, 2)
        print(f"    Obj#{tid}: Δ={delta:+.3f}m, cam={cam_move:.3f} → {state}")

    prev_depths = cur_depths

    # ---------------------------
    # 프레임 시각화 (matplotlib)
    # ---------------------------
    plt.figure(figsize=(8, 6))
    plt.imshow(cv2.cvtColor(color, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.title(f"Frame {frame_count}")
    plt.show()

    frame_count += 1
    return cv2.waitKey(1)


# ---------------------------
# 메인 루프
# ---------------------------
try:
    while True:
        frames = pipeline.wait_for_frames()
        if process_frame(frames) == 27:
            break
finally:
    pipeline.stop()
    cv2.destroyAllWindows()
    print("✅ 모든 프레임 표시 완료")